## Setup

### Remove notebook Warning

In [16]:
import warnings
warnings.filterwarnings('ignore')

### Auto-reloard and better matplot

In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [18]:
import cv2
import cvzone
from ultralytics import YOLO
import time
from IPython.display import Video
from screeninfo import get_monitors
import os

### Monitor Infos

In [19]:
# Get a list of connected monitors
monitors = get_monitors()

# Get the dimensions of the first monitor
monitor = monitors[0]
monitor_width = monitor.width
monitor_height = monitor.height

## Params

In [20]:
video_name = "P1077418_Balcon_4K25FPS.MP4"

mask_name = "P1077418_Balcon_4K25FPS_MASK.jpg"
apply_mask = True
display_mask = True

frame_by_frame = True


## Main

### Model

In [21]:
facemodel_path = os.path.join(os.path.dirname(os.getcwd()), "Model/yolov8n-face.pt")
if os.path.exists(facemodel_path):
    facemodel = YOLO(facemodel_path)
    pass
else :
    raise FileExistsError


### Loading file

In [22]:
#Path video to analyse and check file exist in ./Videos/video_name
file_video_path = os.path.join(os.path.dirname(os.getcwd()), "Videos", video_name )
if os.path.exists(file_video_path):
    pass
else :
    raise FileExistsError

### Open Video

In [23]:
cap = cv2.VideoCapture(file_video_path)

### Video Infos

In [24]:
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps)


video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

### Open Mask

In [25]:
if apply_mask :

    mask_path = os.path.join(os.path.dirname(os.getcwd()), "Videos", mask_name )

    if os.path.exists(mask_path):
        mask = cv2.imread(mask_path)

        mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        _, mask_binary = cv2.threshold(mask_gray, 128, 255, cv2.THRESH_BINARY)

    else:
        raise FileExistsError


### Display Video and detection

In [29]:
# Create a window with the size of the video at the calculated position
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
cv2.moveWindow("Image", (video_width - monitor_width) // 2, (video_height - monitor_height) // 2)
cv2.resizeWindow("Image", monitor_width, monitor_height)

frame_number = 0
while cap.isOpened():



    success, img = cap.read()
    if success:
        frame_number += 1

        if apply_mask:
            img_prepredict = cv2.bitwise_and(img, img, mask=mask_binary)
            if display_mask:
                img = img_prepredict
            else :
                pass

        else :
            img_prepredict = img


        face_result = facemodel.predict(img_prepredict, conf=0.40)
        for info in face_result:
            parameters = info.boxes
            for box in parameters:
                x1,y1,x2,y2 = box.xyxy[0]
                x1,y1,x2,y2 = int(x1), int(y1), int(x2), int(y2)
                h,w = y2 - y1, x2 - x1
                cvzone.cornerRect(img, [x1,y1,w,h], l=9, rt=3)



        #display FPS
        cv2.putText(img, f'FPS: {int(fps)}', (10  , video_height - 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
        cv2.putText(img, f'Frame : {int(frame_number)}', (10  , video_height - 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)




        cv2.imshow("Image", img)








        if frame_by_frame:
            key = cv2.waitKey(0) & 0xFF
            # if the 'q' key is pressed, exit from loop
            if key == ord("q"):
                cv2.destroyAllWindows()
                break
            #if the 'n' key is pressed, go to next frame
            if key == ord("n"):
                continue
        else :
            #continue loop
            if cv2.waitKey(delay) == ord('q'):
                cv2.destroyAllWindows()
                break


0: 384x640 4 faces, 26.8ms
Speed: 4.5ms preprocess, 26.8ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 faces, 21.0ms
Speed: 2.7ms preprocess, 21.0ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 faces, 21.6ms
Speed: 6.8ms preprocess, 21.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 faces, 17.9ms
Speed: 3.1ms preprocess, 17.9ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 faces, 18.4ms
Speed: 5.0ms preprocess, 18.4ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 faces, 14.5ms
Speed: 2.9ms preprocess, 14.5ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 faces, 16.4ms
Speed: 3.0ms preprocess, 16.4ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 faces, 17.9ms
Speed: 3.6ms preprocess, 17.9ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)
